In [23]:
from fungsi import S0, S1, M0, M1, m0_mix, m1_mix, f0, f1

## **INITIAL WHITENING KEY**

In [4]:
plaintext = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06,
             0x07, 0x08, 0x09, 0x0a, 0x0b, 0x0c, 0x0d, 0x0e, 0x0f]

key = [0xff, 0xee, 0xdd, 0xcc, 0xbb, 0xaa, 0x99, 0x88,
       0x77, 0x66, 0x55, 0x44, 0x33, 0x22, 0x11, 0x00]

In [5]:
def whitening_xor(state1, state2):
    temp_xor = []
    for i in range(4):
        temp_xor.append(state1[i] ^ state2[i])
    return temp_xor


ehx = whitening_xor(plaintext[4:8], key[0:4])
final = [hex(x)[2:] for x in ehx]

print(final)

['fb', 'eb', 'db', 'cb']


In [6]:
# whitening key
# initial whitening key

# plain    = 00010203 04050607 08090a0b 0c0d0e0f
# key      = ffeeddcc bbaa9988 77665544 33221100
# after    = 00010203 fbebdbcb 08090a0b b7a79787

def wkey_initial(plain, key):
    wk = plain[0:4] + whitening_xor(plain[4:8], key[0:4]) + \
        plain[8:12] + whitening_xor(plain[12:16], key[4:8])
    return wk

In [7]:
initial_white_key = wkey_initial(plain=plaintext, key=key)

initial_wk = [hex(x)[2:] for x in initial_white_key]
print(initial_wk)
print("--------------")
print(len(initial_wk))

['0', '1', '2', '3', 'fb', 'eb', 'db', 'cb', '8', '9', 'a', 'b', 'b7', 'a7', '97', '87']
--------------
16


## **GFN (d,r)**  
GFN_{d,r}  

> d : branch  
r   : round  
Clefia-128 memiliki 4 branch tiap round.  
menjadi GFN_{4,r}  

>GFN menggunakan 2 fungsi, F0 dan F1 yang masing - masing 32bit  
F0 berada pada data line pertama  
F1 berada pada data line ketiga  

In [8]:
def matriks(mat):
    matriks = []
    for i in range(4):
        four_matriks = mat[i*4: i*4+4]
        matriks.append(four_matriks)
    return matriks

In [9]:
tes = matriks(initial_wk)

In [10]:
print(tes)
print(tes[0])

[['0', '1', '2', '3'], ['fb', 'eb', 'db', 'cb'], ['8', '9', 'a', 'b'], ['b7', 'a7', '97', '87']]
['0', '1', '2', '3']


In [17]:
X0, X1, X2, X3 = tes[0], tes[1], tes[2], tes[3]

X0 = [int(i, base=16) for i in X0]
X1 = [int(i, base=16) for i in X1]
X2 = [int(i, base=16) for i in X2]
X3 = [int(i, base=16) for i in X3]

In [18]:
print(X0)
print(X1)
print(X2)
print(X3)

[0, 1, 2, 3]
[251, 235, 219, 203]
[8, 9, 10, 11]
[183, 167, 151, 135]


In [19]:
rk0 = [243, 230, 206, 249]
rk1 = [141, 247, 94, 56]

In [24]:
lane1 = f0(X0, rk0)
print(f"f0 after M (round-0): {lane1}")
lane1_hex = [hex(i)[2:] for i in lane1]
print(f"f0_hex after M (round-0): {lane1_hex}")

f0 after M (round-0): [84, 122, 49, 147]
f0_hex after M (round-0): ['54', '7a', '31', '93']


In [25]:
lane3 = f1(X2, rk1)
print(f"f1 after M (round-0): {lane3}")
lane3_hex = [hex(i)[2:] for i in lane3]
print(f"f1_hex after M (round-0): {lane3_hex}")

f1 after M (round-0): [171, 241, 32, 112]
f1_hex after M (round-0): ['ab', 'f1', '20', '70']


In [ ]:
# Diffusion Matrices
# y = M0 trans(T) adalah
def M0_trans(T0, T1, T2, T3):
    y0 = T0 ^ (0x02 * T1) ^ (0X04 * T2) ^ (0X06 * T3)
    y1 = (0X02 * T0) ^ (T1) ^ (0X06 * T2) ^ (0X04 * T3)
    y2 = (0X04 * T0) ^ (0X06 * T1) ^ (T2) ^ (0X02 * T3)
    y3 = (0X06 * T0) ^ (0X04 * T1) ^ (0X02 * T2) ^ (T3)
    return y0, y1, y2, y3

# y = M1 trans(T) adalah


def M1_trans(T0, T1, T2, T3):
    y0 = T0 ^ (0x08 * T1) ^ (0x02 * T2) ^ (0x0a * T3)
    y1 = (0x08 * T0) ^ T1 ^ (0x0a * T2) ^ (0x02 * T3)
    y2 = (0x02 * T0) ^ (0x0a * T1) ^ T2 ^ (0x08 * T3)
    y3 = (0x0a * T0) ^ (0x02 * T1) ^ (0x08 * T2) ^ T3
    return y0, y1, y2, y3

In [ ]:
# input     : 32-bit round key RK, 32-bit data x,
# output    : 32-bit data y

def F0(rk, x):
    T = rk ^ x
    T0 = S0[T0]
    T1 = S1[T1]
    T2 = S0[T2]
    T3 = S1[T3]
    # T = T0 + T1 + T2 + T3
    y0, y1, y2, y3 = M0_trans(T0, T1, T2, T3)
    y = y0 + y1 + y2 + y3
    return y


def F1(rk, x):
    T = rk ^ x
    T0 = S1[T0]
    T1 = S0[T1]
    T2 = S1[T2]
    T3 = S0[T3]
    # T = T0 + T1 + T2 + T3
    y0, y1, y2, y3 = M1_trans(T0, T1, T2, T3)
    y = y0 + y1 + y2 + y3
    return y

## **DATA PROCESSING PART**

***ENC***

In [ ]:
# T0,T1,T2,T3 = P0 + (P1 ^ WK0) + P2 + (P3 ^ WK1)
# P = plaintext
# C = ciphertext
# WK = whitening key

# step 1
T0 = P0
T1 = P1 ^ WK0
T2 = P2
T3 = P3 ^ WK1

# step 2
GFN_{4,r}(RK_{0}, ..., RK_{2r - 1}, T0, T1, T2, T3)

# step 3
C0 = T0
C1 = T1 ^ WK2
C2 = T2
C3 = T3 ^ WK3


## **FINAL WHITENING KEY**

In [63]:
state_akhir = [0xde, 0x2b, 0xf2, 0xfd, 0xec, 0x12, 0xff,
               0x89, 0xf1, 0x29, 0x85, 0x55, 0x76, 0xb6, 0x85, 0xfd]

In [64]:
def final_whitening_key(state, key):
    final_wk = state[0:4] + whitening_xor(state[4:8], key[8:12]) + \
        state[8:12] + whitening_xor(state[12:16], key[12:16])
    return final_wk

In [66]:
final_white_key = final_whitening_key(state=state_akhir, key=key)
final_wk = [hex(x)[2:] for x in final_white_key]
print(final_wk)
print(len(final_wk))

['de', '2b', 'f2', 'fd', '9b', '74', 'aa', 'cd', 'f1', '29', '85', '55', '45', '94', '94', 'fd']
16
